In [1]:
import torch
import matplotlib.pyplot as plt
import yousef_model as model
import Naicheng.res.process_data.process_output as pro_out
import Naicheng.res.process_data.process_raw_data as pr
import Naicheng.res.process_data.dire_and_coor as dc
from Naicheng.res.process_data.dataset import tensor_dataset
import Naicheng.res.process_data.loop_overlap as lp
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
import json

In [4]:
epoch = 4000
name = f'models/yousef_gan_{epoch}.pth.tar'
gen = model.GeneratorNet()
check_point = torch.load(name)
gen.load_state_dict(check_point['gen_state_dict'])
num_ite = int(100000 // 8 * 1.13)
output_direction = np.zeros([num_ite * 8, 15, 1])
for j in range(num_ite):
    # get output
    noise = model.noise(8)
    output = gen(noise).data.numpy()
    # slice the output to a (n ,15 ,1)
    output_direction[8 * j : 8 * (j + 1)] = pro_out.slice_directions(output)
output_coor = dc.direction_coordinate_three(output_direction)
loop_all = []
for k in range(len(output_coor)):
    if lp.check_fold_cross(output_coor[k]):
        loop_all += lp.loop_length(output_coor[k])